# Trains and Tests PLDA

In [115]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
sys.path.insert(0, "/home/sjoshi/codes/python/BeatPD/code")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error

#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Confusion matrix
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [116]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 exp	 newpldatest.ipynb  run3ax.sh	run.sh	 utils
cmd.sh~  exp3ax  path.sh	    run3ax.sh~	run.sh~
conf	 local	 pca_pldapy.ipynb   runFor.sh	sid
data	 mfcc	 README.txt	    runFor.sh~	steps
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 exp	 newpldatest.ipynb  run3ax.sh	run.sh	 utils
cmd.sh~  exp3ax  path.sh	    run3ax.sh~	run.sh~
conf	 local	 pca_pldapy.ipynb   runFor.sh	sid
data	 mfcc	 README.txt	    runFor.sh~	steps


In [5]:
# def pca(sFileTrai, sFileTest, iComponents):
# # os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# # sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# # sOut = Folder where to store results 
# # iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
#     dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
#     vTrai= pd.DataFrame((list(dIvecTrai.values())))
#     vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
#     # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
#     # FIXME REAL-PD it's not only int 
#     vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

#     dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
#     vTest=np.array(list(dIvecTest.values()), dtype=float)
#     vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
#     vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])

#     print('before transform : ', vTrai.shape)
#     print('before transform : ', vTest.shape)

#     if isinstance(iComponents, str):
#             iComponents=int(iComponents)

#     pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
#     pca.fit(vTrai)


#     vTraiPCA=pca.transform(vTrai)
#     vTestPCA=pca.transform(vTest)

#     print('after transform : ', vTraiPCA.shape)
#     print('after transform : ', vTestPCA.shape)
#     return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId

In [117]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
from pca_knn_bpd import pca_knn_bpd

In [120]:
from pca_plda_bpd import pca_plda_bpd

In [ ]:
pca_knn_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents, 3)

In [113]:
pca_plda_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents)

----- 1004----- 
Previous shapes:
(63, 50)
(63,)
Shapes after removing classes:
(63, 50)
(63,)
Accuracy: 0.2631578947368421
----- 1006----- 
Previous shapes:
(28, 50)
(28,)
Shapes after removing classes:
(28, 50)
(28,)
Accuracy: 0.6666666666666666
----- 1007----- 
Previous shapes:
(219, 50)
(219,)
Shapes after removing classes:
(219, 50)
(219,)
Accuracy: 0.543859649122807
----- 1019----- 
Previous shapes:
(35, 50)
(35,)
Shapes after removing classes:
(35, 50)
(35,)
Accuracy: 0.3
----- 1020----- 
Previous shapes:
(154, 50)
(154,)
Shapes after removing classes:
(154, 50)
(154,)
Accuracy: 0.7804878048780488
----- 1023----- 
Previous shapes:
(82, 50)
(82,)
Shapes after removing classes:
(82, 50)
(82,)
Accuracy: 0.20833333333333334
----- 1032----- 
Previous shapes:
(139, 50)
(139,)
Shapes after removing classes:
(139, 50)
(139,)
Accuracy: 0.3157894736842105
----- 1034----- 
Previous shapes:
(30, 50)
(30,)
Shapes after removing classes:
(30, 50)
(30,)
Accuracy: 0.2
----- 1038----- 
Previous 

In [575]:
def final_score(mse_per_subjectid, nb_files_per_subject_id):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
    print('numerator : ', numerator)
    print('denominator : ', denominator)
    print('Final score : ', np.divide(numerator, denominator))
    
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

In [556]:
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

numerator :  2.9956883253194277
denominator :  138.07717596772696
Final score :  0.021695753149091168
numerator :  6.065415875730853
denominator :  70.2868377625761
Final score :  0.08629518795851071


In [543]:
def do_confusion_matrix(y_test, predictions):
    print('y test : ', np.unique(y_test))
    LABELS_NEW = np.unique(y_test)
    n_classes=np.unique(y_test)
    print("Confusion Matrix:")
    confusion_matrix = metrics.confusion_matrix(y_test, predictions)
    print(confusion_matrix)
    normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

    print("")
    print("Confusion matrix (normalised to % of total test data):")
    print(normalised_confusion_matrix)

    # Plot Results:
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.imshow(
        normalised_confusion_matrix,
        interpolation='nearest',
        cmap=plt.cm.rainbow
    )
    plt.title("Confusion matrix \n(normalised to % of total test data)")
    plt.colorbar()
    tick_marks = np.arange(n_classes)
    plt.xticks(tick_marks, LABELS_NEW, rotation=90)
    plt.yticks(tick_marks, LABELS_NEW)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()